# Time

One of the challenges of working with spreadsheet data is that spreadsheets have specific handling of time that doesn't always translate well when parsed into python.

In all cases tidychef will seek to format the time type cells via the exact same human readable format said cells would appear as when viewed via excel.

The process is as follows:

- When an excel cell is read that has an _excel type_ of time.
- tidychef will check the excel information for _how that time type is formatted for presentation_.
- tidychef will apply that formatting so the value in tidychef **matches that you'd expect from viewing the original source**.
- if _custom_ or unknown time formatting is applied, tidychef will fall back on the raw cell value and will show a warning.

An example of this behaviour and how you can handle these scenarios is detailed below.

In [4]:
from typing import List

from tidychef import acquire, preview
from tidychef.selection import XlsSelectable

tables: List[XlsSelectable] = acquire.xls.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/house-prices.xls")

For more details on handling excel time formatting see tidychef documentation. Cell(s) in question (max 5 shown): x:2, y:13


Now lets have a look at the cell in question

In [5]:
table: XlsSelectable = acquire.xls.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/house-prices.xls",
                                               tables="Contents")
suspect_cell = table.excel_ref("C14")
preview(suspect_cell, show_xy=True)

For more details on handling excel time formatting see tidychef documentation. Cell(s) in question (max 5 shown): x:2, y:13


You'll notice it just contains a number, that's a python [datetime](https://docs.python.org/3/library/datetime.html) object representing the specific moment in time in a way that python understands.

In this instance someone has created quite a complex custom time formatting string of `[$-F800]dddd\,\ mmmm\ dd\,\ yyyy` to represent the value (as expressed in the original file) as:

```
Wednesday, August 16th, 2023
````

So how do we specify the time formatting to use for these kinds of custom and complex time formats?

## Specifying time formatting



Tidychef allows you to map excel formats to the standard patterns supported by the python [datatime.strftime()](https://www.programiz.com/python-programming/datetime/strftime) method.

This is done by passing an appropriate dictionary into acquire with the keyword argument `custom_time_formats=` to map excel time formats to a `strftime()` pattern.

In this instance we'll provide the `stftime()` format string for the excel pattern in question.

Example follows:

In [6]:
from tidychef import acquire, preview
from tidychef.selection import XlsSelectable

table: XlsSelectable = acquire.xls.http("https://raw.githubusercontent.com/mikeAdamss/tidychef/main/tests/fixtures/xls/house-prices.xls",
                                        custom_time_formats={"[$-F800]dddd\,\ mmmm\ dd\,\ yyyy": "%A, %B %d, %Y"}, tables="Contents")
suspect_cell = table.excel_ref("C14")
preview(suspect_cell, show_xy=True)

## Note

tidychef comes with a large number of excel format to `strftime()` translations as default but given the high level of customization an exhaustive mapping is difficult to create.

Users are encouraged to PR additional mappings as encountered to [this file](https://github.com/mikeAdamss/tidychef/blob/main/tidychef/acquire/excel_time.py). 